In [1]:
import requests
from bs4 import BeautifulSoup
from splinter import Browser


In [2]:
browser = Browser('chrome')



In [44]:
url = 'https://ca.indeed.com/q-junior-data-analyst-l-toronto,-on-jobs.html?from=relatedQueries&saIdx=2&rqf=1&parentQnorm=data+analyst+entry+level'
browser.visit(url)
html = browser.html

soup = BeautifulSoup(html, 'html.parser')



In [45]:
html

'<html dir="ltr" lang="en" class="js-focus-visible" data-js-focus-visible=""><head>\n    <link rel="shortcut icon" href="/images/favicon.ico">\n    <title>Junior Data Analyst Jobs in Toronto, ON (with Salaries) 2023 | Indeed.com Canada</title>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n    <meta name="description" content="Search 17 Junior Data Analyst jobs now available in Toronto, ON on Indeed.com, the world\'s largest job site.">\n    <meta name="keywords" content="Junior Data Analyst Jobs in Toronto, ON, Toronto, ON careers, Toronto, ON employment, Toronto, ON job listings, Toronto, ON job search, Toronto, ON search engine, work in Toronto, ON">\n    <meta name="referrer" content="origin-when-cross-origin">\n    <link rel="canonical" href="https://ca.indeed.com/q-junior-data-analyst-l-toronto,-on-jobs.html">\n    <link rel="alternate" href="android-app://com.indeed.android.jobsearch/https/ca.indeed.com/m/jobs?from=relatedQueries&amp;saIdx=2&amp;rqf=1&

In [46]:
job_info = soup.find_all('li')

In [49]:
type(job_info)

bs4.element.ResultSet

In [38]:
job_info

[<li class="yosegi-FilterPill-dropdownListItem" role="menuitem" tabindex="-1"><a class="yosegi-FilterPill-dropdownListItemLink" href="/jobs?q=junior+data+analyst&amp;l=toronto%2C+on&amp;fromage=1" tabindex="-1">Last 24 hours</a></li>,
 <li class="yosegi-FilterPill-dropdownListItem" role="menuitem" tabindex="-1"><a class="yosegi-FilterPill-dropdownListItemLink" href="/jobs?q=junior+data+analyst&amp;l=toronto%2C+on&amp;fromage=3" tabindex="-1">Last 3 days</a></li>,
 <li class="yosegi-FilterPill-dropdownListItem" role="menuitem" tabindex="-1"><a class="yosegi-FilterPill-dropdownListItemLink" href="/jobs?q=junior+data+analyst&amp;l=toronto%2C+on&amp;fromage=7" tabindex="-1">Last 7 days</a></li>,
 <li class="yosegi-FilterPill-dropdownListItem" role="menuitem" tabindex="-1"><a class="yosegi-FilterPill-dropdownListItemLink" href="/jobs?q=junior+data+analyst&amp;l=toronto%2C+on&amp;fromage=14" tabindex="-1">Last 14 days</a></li>,
 <li class="yosegi-FilterPill-dropdownListItem" role="menuitem" 

In [47]:
job_titles = []
for job in job_info:
    title = job.find('a').get_text()
    company = job.find('div',class_='companyName')
    location = job.find('div',class_='companyLocation')
    

    job_dict = {
        'Title':title,
        'Company':company,
        'Location':location
    }
    job_titles.append(job_dict)

print(job_titles)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
<a id="sj_5cbb667ba6b68837" data-mobtk="1gr15uh51k7ep800" data-jk="5cbb667ba6b68837" data-ci="386987198" data-empn="653442265499938" data-hiring-event="false" target="_blank" data-hide-spinner="true" role="button" aria-label="full details of Junior Logistics Data Analyst" class="jcs-JobTitle css-jspxzf eu4oa1w0" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0DetcgDfGjoU1pTmQy41gfWat4k6tnLCAOKfyt8FccB-ME3X-V7aTd9RjI2D8j2GuvwsDv2FCQpC2lHPY-VYu2usy57mLFJUXh6hq8jtecqdQDsvH6luzSBxbDHy2UiGSU62HJLCxsqBDdU-p90n2-gHX__kegiH5aG4DLEKz5SO0h9X496yAlz-JjIwRkL6l5TQuNMt4nGdgotO-UqjLEpbGpXTHNRMj6rclicJ-QQ0Fp62yi0NyVqeQK3U56_kgmzVeiKRjthCA1znthCEq5AyoTgwaifW4eJCC-kvcKBUVqHhoKk3X-AUVh7RtCXseJ7eTJ-2y00vwaczRN4yx7g5pwt2XwRB3JoxKYhlX4H-7hGWwSmecEa__e7x7LK4PCypNghh4npUj61zKMSaF8gnr12P8Rg53RLgVQfcZhrXy5wivXH_Qdq7tfRAPtMjGX6WoO5RWBZu-bBfKzkX_8qpsp96POheY8dTt78xFhqb9B_998BeRoWo_V_56SdVBDeHvUa7ALaN1j1ewYhKL8qDkd6PML6obNpw_Qt1EToGMyUeCGEAL32qkxjjcyXwIFwLm7X_-lWRiz1Zbdqi1pYDBlKWPYMLQZ8tkILp05LK2eqYLJwsIk6p8kPfIOWbPcS8m8Pp454wIrp1FfOi3PTfBcC9KiBvjQhcZOq49i-9oE40vgX8YN_cycnbE25DQYRjGGxOKflbARMQAKXoJzsNomCUjLblbrGaMgekmExdphBL3R3emiUa8Vr9ZmbylusFsjCQIJl92Fa0NQ3bkDE73p9L-4Sr3SO3eoGYLUg08lVS4A-0288cVdCkYKg_t2Ew9QI1WxyzA==&amp;xkcb=SoCE-_M3ShD1_P2koR0LbzkdCdPP&amp;p=0&amp;fvj=0&amp;vjs=3"><span title="Junior Logistics Data Analyst" id="jobTitle-5cbb667ba6b68837">Junior Logistics Data Analyst</span></a>

In [24]:
url = 'https://ca.indeed.com/q-junior-data-analyst-l-toronto,-on-jobs.html?from=relatedQueries&saIdx=2&rqf=1&parentQnorm=data+analyst+entry+level'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

job_listings = soup.find_all('div', class_='jobsearch-SerpJobCard')

for job in job_listings:
    # extract job title
    job_title = job.select_one('a').get_text()

    # extract company name
    company_name = job.select_one('.company').get_text().strip()

    # extract location
    location = job.select_one('.location').get_text().strip()

    print(f'Job Title: {job_title}')
    print(f'Company: {company_name}')
    print(f'Location: {location}')
    print('---')

In [50]:
job_list = soup.find_all('li', class_='result')

for job in job_list:
    title_element = job.find('span', class_='title')
    if title_element is not None:
        title = title_element.text.strip()
        print('Job Title:', title)

    company_element = job.find('span', class_='company')
    if company_element is not None:
        company = company_element.text.strip()
        print('Company:', company)

    location_element = job.find('span', class_='location')
    if location_element is not None:
        location = location_element.text.strip()
        print('Location:', location)

    print('-' * 50)